In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from keras.layers import Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
import datetime
import time

In [ ]:
!gsutil cp gs://gtsrb_raw_final/GTSRB_Final_Training_Images.zip .
!unzip GTSRB_Final_Training_Images.zip

In [2]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
        'GTSRB/Final_Training/Images',
        target_size=(100, 100),
        batch_size=64,
        class_mode='categorical',
        subset='training')

val_generator = datagen.flow_from_directory(
        'GTSRB/Final_Training/Images',
        target_size=(100, 100),
        batch_size=64,
        class_mode='categorical',
        subset='validation')

Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [19]:
inception_resnet = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(100, 100, 3),
    classifier_activation="softmax",
)

In [20]:
model = models.Sequential()
model.add(inception_resnet)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=43, activation='softmax'))

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 1, 1, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten_1 (Flatten)         (None, 1536)              0         
                                                                 
 dropout_1 (Dropout)         (None, 1536)              0         
                                                                 
 dense_1 (Dense)             (None, 43)                66091     
                                                                 
Total params: 54,402,827
Trainable params: 54,342,283
Non-trainable params: 60,544
_________________________________________________________________


In [2]:
%load_ext tensorboard

In [23]:
log_dir = "logs_inc_resnet/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
steps_per_epoch = 800

optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [25]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [26]:
tic = time.time()
model.fit(train_generator, validation_data=val_generator, epochs = 20, callbacks=[es,tensorboard_callback])
toc = time.time()

/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20
491/491 [==============================] - 142s 253ms/step - loss: 0.0154 - accuracy: 0.9165 - val_loss: 0.0140 - val_accuracy: 0.9120
Epoch 2/20
491/491 [==============================] - 123s 251ms/step - loss: 0.0014 - accuracy: 0.9926 - val_loss: 0.0058 - val_accuracy: 0.9656
Epoch 3/20
491/491 [==============================] - 125s 253ms/step - loss: 0.0010 - accuracy: 0.9947 - val_loss: 0.0066 - val_accuracy: 0.9606
Epoch 4/20
491/491 [==============================] - 125s 255ms/step - loss: 8.0823e-04 - accuracy: 0.9962 - val_loss: 0.0035 - val_accuracy: 0.9753
Epoch 5/20
491/491 [==============================] - 126s 255ms/step - loss: 9.8102e-04 - accuracy: 0.9947 - val_loss: 0.0022 - val_accuracy: 0.9861
Epoch 6/20
491/491 [==============================] - 126s 255ms/step - loss: 8.0738e-04 - accuracy: 0.9957 - val_loss: 0.0035 - val_accuracy: 0.9773
Epoch 7/20
491/491 [==============================] - 126s 255ms/step - loss: 6.6488e-04 - accuracy: 0.9964 - va

In [29]:
print('Time taken for training: {} '.format(toc-tic))

Time taken for training: 2521.969240427017 


In [ ]:
%tensorboard --logdir logs_inc_resnet/fit

In [32]:
for layer in inception_resnet.layers:
    layer.trainable = False
model1 = models.Sequential()
model1.add(inception_resnet)
model1.add(Flatten())
model1.add(Dropout(0.5))
model1.add(Dense(units=43, activation='softmax'))
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 1, 1, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten_3 (Flatten)         (None, 1536)              0         
                                                                 
 dropout_3 (Dropout)         (None, 1536)              0         
                                                                 
 dense_3 (Dense)             (None, 43)                66091     
                                                                 
Total params: 54,402,827
Trainable params: 66,091
Non-trainable params: 54,336,736
_________________________________________________________________


In [33]:
log_dir = "logs_inc_resnet_NT/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [34]:
steps_per_epoch = 729

optimizer = tf.keras.optimizers.Adam()

model1.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [35]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [36]:
tic = time.time()
model1.fit(train_generator, validation_data=val_generator, epochs = 10, callbacks=[es,tensorboard_callback])
toc = time.time()

/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/10
491/491 [==============================] - 108s 202ms/step - loss: 0.0164 - accuracy: 0.9494 - val_loss: 0.0040 - val_accuracy: 0.9889
Epoch 2/10
491/491 [==============================] - 96s 196ms/step - loss: 9.0343e-04 - accuracy: 0.9996 - val_loss: 0.0028 - val_accuracy: 0.9874
Epoch 3/10
491/491 [==============================] - 97s 197ms/step - loss: 4.6670e-04 - accuracy: 0.9997 - val_loss: 0.0023 - val_accuracy: 0.9888
Epoch 4/10
491/491 [==============================] - 97s 197ms/step - loss: 3.0562e-04 - accuracy: 0.9997 - val_loss: 0.0021 - val_accuracy: 0.9876
Epoch 5/10
491/491 [==============================] - 96s 196ms/step - loss: 2.1855e-04 - accuracy: 0.9999 - val_loss: 0.0018 - val_accuracy: 0.9888
Epoch 6/10
491/491 [==============================] - 96s 196ms/step - loss: 1.7390e-04 - accuracy: 0.9999 - val_loss: 0.0018 - val_accuracy: 0.9876
Epoch 7/10
491/491 [==============================] - 96s 195ms/step - loss: 1.1915e-04 - accuracy: 0.9998 - 

In [37]:
print('Time taken for training: {} '.format(toc-tic))

Time taken for training: 976.0215952396393 


In [ ]:
%tensorboard --logdir logs_inc_resnet_NT/fit